This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [1]:
import os
import urllib
from pathlib import Path
from zipfile import ZipFile

from pydrake.all import ModelVisualizer, PackageMap, Simulator, StartMeshcat
from tqdm.notebook import tqdm

from manipulation.make_drake_compatible_model import MakeDrakeCompatibleModel
from manipulation.utils import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
def AddRobocasaRemote(package_map):
    package_map.AddRemote(
        package_name="robocasa",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/robocasa/robocasa/archive/a7586f67b72e51722340c9dbe97a59e0aff1ff8f.zip"
            ],
            sha256=("64abe91ac8ca9cbb22aa4d7c9461d7d899ba54e700dccecdfab5a827fcf7f322"),
            strip_prefix="robocasa-a7586f67b72e51722340c9dbe97a59e0aff1ff8f/robocasa/",
        ),
    )


def MaybeDownloadRobocasaKitchenAssets():
    package_map = PackageMap()
    AddRobocasaRemote(package_map)
    # This will force the download if it hasn't been done before.
    robocasa_path = package_map.GetPath("robocasa")

    # The asset registry dictionary is copied from
    # https://github.com/robocasa/robocasa/blob/main/robocasa/scripts/download_kitchen_assets.py
    # with robocasa_path updated.
    DOWNLOAD_ASSET_REGISTRY = dict(
        textures=dict(
            message="Downloading environment textures",
            url="https://utexas.box.com/shared/static/otdsyfjontk17jdp24bkhy2hgalofbh4.zip",
            folder=os.path.join(robocasa_path, "models/assets/textures"),
            check_folder_exists=True,
        ),
        fixtures=dict(
            message="Downloading fixtures",
            url="https://utexas.box.com/shared/static/pobhbsjyacahg2mx8x4rm5fkz3wlmyzp.zip",
            folder=os.path.join(robocasa_path, "models/assets/fixtures"),
            check_folder_exists=True,
        ),
        objaverse=dict(
            message="Downloading objaverse objects",
            url="https://utexas.box.com/shared/static/ejt1kc2v5vhae1rl4k5697i4xvpbjcox.zip",
            folder=os.path.join(robocasa_path, "models/assets/objects/objaverse"),
            check_folder_exists=True,
        ),
        #  aigen_objs=dict(
        #     message="Downloading AI-generated objects",
        #     url="https://utexas.box.com/shared/static/os3hrui06lasnuvwqpmwn0wcrduh6jg3.zip",
        #     folder=os.path.join(robocasa_path, "models/assets/objects/aigen_objs"),
        #     check_folder_exists=False,
        # ),
        # generative_textures=dict(
        #     message="Downloading AI-generated environment textures",
        #     url="https://utexas.box.com/shared/static/gf9nkadvfrowkb9lmkcx58jwt4d6c1g3.zip",
        #     folder=os.path.join(robocasa_path, "models/assets/generative_textures"),
        #     check_folder_exists=False,
        # ),
    )

    def show_progress(block_num, block_size, total_size):
        if pbar.total is None:
            pbar.total = total_size
        pbar.update(block_size)

    for name, info in DOWNLOAD_ASSET_REGISTRY.items():
        if info["check_folder_exists"] and os.path.exists(info["folder"]):
            print(f"Skipping {name} - already downloaded")
            continue
        with tqdm(unit="B", unit_scale=True, miniters=1, desc=info["message"]) as pbar:
            filename, headers = urllib.request.urlretrieve(
                info["url"], reporthook=show_progress
            )
        folder = Path(info["folder"]).parent
        with ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(path=folder)
        os.remove(filename)


if running_as_notebook:
    MaybeDownloadRobocasaKitchenAssets()

# Browse the assets

First we'll use the ModelVisualizer to inspect the models that are available [here](https://github.com/robocasa/robocasa/tree/1370b9e0f747d84fb21ed29bacefb1654865301b/robocasa).

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
package_map = visualizer.parser().package_map()
AddRobocasaRemote(package_map)
original_model_path = package_map.ResolveUrl(
    "package://robocasa/models/assets/fixtures/accessories/knife_blocks/dark_wood/model.xml"
)
drake_model_path = original_model_path.replace(".xml", ".drake.xml")
MakeDrakeCompatibleModel(
    original_model_path,
    drake_model_path,
    remap_mujoco_geometry_groups={0: 3},
)
visualizer.AddModels(drake_model_path)
visualizer.Run(loop_once=True)
meshcat.DeleteAddedControls()